In [ ]:
!pip install transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=14fabbed2d8270ede9051d564cde5796cce45fe82376e5756c30ecadcbca0c11
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:


import json
import pandas as pd
import numpy as np
import re
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [ ]:
print("Loading data from resume_data.json...")

try:
    with open('/content/resume_data.json', 'r', encoding='utf-8') as f:
        file_content = f.read()

    lines = file_content.split('\n')

    data = []
    for line in lines:
        if line.strip():  # Skip empty lines
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON: {str(e)} - {line[:50]}...")

    print(f"Successfully loaded {len(data)} examples")

except Exception as e:
    print(f"Error loading file: {str(e)}")
    data = []

Loading data from resume_data.json...
Successfully loaded 430 examples


In [ ]:

# Data exploration
if data:
    example = data[0]
    print("\nExample content (truncated to 200 chars):")
    print(example['content'][:200])
    print("\nExample annotations:")
    for annotation in example['annotation']:
        print(f"Label: {annotation['label']}")
        for point in annotation['points']:
            print(f"  - Text: {point['text'][:50]}...")
else:
    print("No data available. Please check file path and format.")


Example content (truncated to 200 chars):
Abhishek Jha
Application Development Associate - Accenture

Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a

• To work for an organization which provides me the o

Example annotations:
Label: ['SKILLS']
  - Text: 
• Programming language: C, C++, Java
• Oracle Peo...
Label: ['SKILLS']
  - Text: C (Less than 1 year), Database (Less than 1 year),...
Label: ['COLLEGE NAME']
  - Text: Kendriya Vidyalaya...
Label: ['COLLEGE NAME']
  - Text: Woodbine modern school...
Label: ['COLLEGE NAME']
  - Text: B.v.b college of engineering and technology...
Label: ['DESIGNATION']
  - Text: B.E in Information science and engineering
...
Label: ['COMPANY']
  - Text: Accenture...
Label: ['DESIGNATION']
  - Text: Application Development Associate...
Label: ['EMAIL']
  - Text: Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a...
Label: ['LOCATION']
  - Text: Bengaluru...
Label: ['COMPANY']
  - Text: Accenture...
Label: ['DESIGN

In [ ]:

# Define unique labels
all_labels = set()
for example in data:
    for annotation in example['annotation']:
        all_labels.update(annotation['label'])

print("\nUnique labels found in the dataset:")
unique_labels = sorted(list(all_labels))
print(unique_labels)


Unique labels found in the dataset:
['COLLEGE NAME', 'COMPANY', 'DEGREE', 'DESIGNATION', 'EMAIL', 'LOCATION', 'NAME', 'SKILLS']


In [ ]:

# Add 'O' for non-entity tokens and special tokens
labels = ['O'] + [f'B-{label}' for label in unique_labels] + [f'I-{label}' for label in unique_labels]
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

print(f"\nTotal number of labels: {len(labels)}")
print(f"Labels: {labels}")


Total number of labels: 17
Labels: ['O', 'B-COLLEGE NAME', 'B-COMPANY', 'B-DEGREE', 'B-DESIGNATION', 'B-EMAIL', 'B-LOCATION', 'B-NAME', 'B-SKILLS', 'I-COLLEGE NAME', 'I-COMPANY', 'I-DEGREE', 'I-DESIGNATION', 'I-EMAIL', 'I-LOCATION', 'I-NAME', 'I-SKILLS']


In [ ]:


# Improved BIO format conversion
def convert_data_to_bio_format(data):
    """Convert all examples to BIO format with improved entity boundary detection"""
    processed_data = []

    for example in data:
        try:
            content = example['content']
            annotations = example['annotation']

            # Initialize all characters with 'O' tag
            char_labels = ['O'] * len(content)

            # Create a more structured representation of annotations to handle overlaps
            entity_spans = []
            for annotation in annotations:
                for label in annotation['label']:
                    for point in annotation['points']:
                        if 'start' in point and 'end' in point:
                            start, end = point['start'], point['end']
                            if start < len(content) and end <= len(content):
                                entity_spans.append((start, end, label))

            # Sort by start position and then by length (longer spans first for overlaps)
            entity_spans.sort(key=lambda x: (x[0], x[1]-x[0]), reverse=True)

            # Apply labels to character sequence
            for start, end, label in entity_spans:
                char_labels[start] = f'B-{label}'
                for i in range(start + 1, end):
                    char_labels[i] = f'I-{label}'

            # Tokenize content into words more carefully
            # This regex splits by whitespace but keeps punctuation separate
            tokens = []
            spans = []

            # Using regex to tokenize while preserving token positions
            for match in re.finditer(r'\S+', content):
                token = match.group()
                start, end = match.span()
                tokens.append(token)
                spans.append((start, end))

            # Assign labels to tokens based on character labels
            token_labels = []
            for token, (start, end) in zip(tokens, spans):
                # Get the labels for this span
                span_labels = char_labels[start:end]

                # Count the labels in this span
                label_counts = Counter(span_labels)

                # Prioritize B- labels, then I- labels, then O
                b_labels = [l for l in span_labels if l.startswith('B-')]
                i_labels = [l for l in span_labels if l.startswith('I-')]

                if b_labels:
                    # Use the most frequent B- label
                    most_common_b = Counter(b_labels).most_common(1)[0][0]
                    token_labels.append(most_common_b)
                elif i_labels:
                    # Use the most common I- label
                    most_common_i = Counter(i_labels).most_common(1)[0][0]
                    token_labels.append(most_common_i)
                else:
                    # No entity
                    token_labels.append('O')

            processed_data.append({
                'tokens': tokens,
                'labels': token_labels,
                'original_text': content
            })

        except Exception as e:
            print(f"Error processing example: {str(e)}")
            continue

    return processed_data

# Process all data
processed_data = convert_data_to_bio_format(data)
print(f"\nProcessed {len(processed_data)} examples to BIO format")


Processed 430 examples to BIO format


In [ ]:

# Show an example of processed data
if processed_data:
    example = processed_data[0]
    print("\nExample processed data (first 10 tokens):")
    for token, label in zip(example['tokens'][:10], example['labels'][:10]):
        print(f"{token} -> {label}")

    # Analyze label distribution
    all_processed_labels = []
    for example in processed_data:
        all_processed_labels.extend(example['labels'])

    label_counts = Counter(all_processed_labels)
    print("\nLabel distribution in processed data:")
    for label, count in label_counts.most_common():
        print(f"{label}: {count}")
else:
    print("No processed data available.")


Example processed data (first 10 tokens):
Abhishek -> B-NAME
Jha -> I-NAME
Application -> B-DESIGNATION
Development -> I-DESIGNATION
Associate -> I-DESIGNATION
- -> O
Accenture -> B-COMPANY
Bengaluru, -> B-LOCATION
Karnataka -> O
- -> O

Label distribution in processed data:
O: 184035
I-SKILLS: 13883
I-COLLEGE NAME: 1698
I-DESIGNATION: 1636
I-DEGREE: 1584
B-COMPANY: 1186
B-DESIGNATION: 928
I-COMPANY: 875
B-SKILLS: 788
B-LOCATION: 783
B-COLLEGE NAME: 612
B-DEGREE: 571
B-EMAIL: 443
B-NAME: 438
I-NAME: 429
I-EMAIL: 186
I-LOCATION: 50


In [ ]:
# STEP 1: Add entity presence info
for example in processed_data:
    present_entities = set()
    for label in example['labels']:
        if label != 'O':
            entity_type = label.split('-')[1]
            present_entities.add(entity_type)
    example['entity_presence'] = '+'.join(sorted(present_entities)) if present_entities else 'none'

# STEP 2: Count combinations
entity_presence_counts = Counter(ex['entity_presence'] for ex in processed_data)
print("\nEntity distribution before split:")
for entity_type, count in entity_presence_counts.most_common():
    print(f"{entity_type}: {count}")


Entity distribution before split:
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 193
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME: 42
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 30
COLLEGE NAME+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 17
COLLEGE NAME+COMPANY+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 14
COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 14
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+NAME+SKILLS: 13
COLLEGE NAME+COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 11
COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 10
COLLEGE NAME+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 7
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+NAME+SKILLS: 7
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME: 6
DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 6
COLLEGE NAME+DEGREE+DESIGNATION+LOCATION+NAME: 5
COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 4
COMPANY+DESIGNATION+EMAIL+LOCATION+NAME: 4
COLLEGE NAME+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 4
COLLE

In [ ]:
# STEP 3: Replace rare combinations with 'OTHER' to retain all data
for example in processed_data:
    if entity_presence_counts[example['entity_presence']] < 2:
        example['entity_presence'] = 'OTHER'

# STEP 4: Final count after merging rare combinations
entity_presence_counts = Counter(ex['entity_presence'] for ex in processed_data)
print("\nEntity distribution after merging rare combinations:")
for entity_type, count in entity_presence_counts.most_common():
    print(f"{entity_type}: {count}")


Entity distribution after merging rare combinations:
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 193
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME: 42
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 30
COLLEGE NAME+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 17
COLLEGE NAME+COMPANY+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 14
COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 14
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+NAME+SKILLS: 13
COLLEGE NAME+COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 11
COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 10
COLLEGE NAME+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 7
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+NAME+SKILLS: 7
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME: 6
DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 6
COLLEGE NAME+DEGREE+DESIGNATION+LOCATION+NAME: 5
COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 4
COMPANY+DESIGNATION+EMAIL+LOCATION+NAME: 4
COLLEGE NAME+DESIGNATION+EMAIL+LOCATION+N

In [ ]:
# STEP 5: Stratified split once (Train vs Temp)
used_stratified = False

try:
    train_data, temp_data = train_test_split(
        processed_data,
        test_size=0.3,
        random_state=42,
        shuffle=True,
        stratify=[ex['entity_presence'] for ex in processed_data]
    )
    used_stratified = True

    val_data, test_data = train_test_split(
        temp_data,
        test_size=0.5,
        random_state=42,
        shuffle=True
    )

except ValueError as e:
    print(f"⚠ Stratified split failed: {e}")
    print("❗ Falling back to full random splitting.")

    train_data, temp_data = train_test_split(
        processed_data, test_size=0.3, random_state=42, shuffle=True
    )
    val_data, test_data = train_test_split(
        temp_data, test_size=0.5, random_state=42, shuffle=True
    )

# STEP 6: Final summary
print("\n✅ Data splitting complete:")
print(f"Train: {len(train_data)}")
print(f"Validation: {len(val_data)}")
print(f"Test: {len(test_data)}")
print("Stratified splitting used ✅" if used_stratified else "Random splitting used ⚠️")



✅ Data splitting complete:
Train: 301
Validation: 64
Test: 65
Stratified splitting used ✅


In [ ]:
# Data Augmentation Functions
import random
def augment_token_replacement(tokens, labels, replacement_prob=0.15):
    """Replace random non-entity tokens with similar length tokens"""
    augmented_tokens = tokens.copy()

    # Create a pool of non-entity tokens from the TRAINING dataset only
    non_entity_tokens = {}
    for length in range(1, 15):  # Group tokens by length
        non_entity_tokens[length] = []

    # Group non-entity tokens by length from training data only
    for example in train_data:  # Use train_data instead of processed_data
        for token, label in zip(example['tokens'], example['labels']):
            if label == 'O':
                length = min(len(token), 14)  # Cap at 14 to avoid sparse groups
                non_entity_tokens[length].append(token)

    # Replace non-entity tokens with probability replacement_prob
    for i, (token, label) in enumerate(zip(tokens, labels)):
        if label == 'O' and random.random() < replacement_prob:
            token_length = min(len(token), 14)
            if non_entity_tokens[token_length]:  # If we have replacements of this length
                replacement = random.choice(non_entity_tokens[token_length])
                augmented_tokens[i] = replacement

    return augmented_tokens, labels

def augment_token_deletion(tokens, labels, deletion_prob=0.05):
    """Delete random non-entity tokens"""
    augmented_tokens = []
    augmented_labels = []

    for token, label in zip(tokens, labels):
        # Only delete O-labeled tokens with a certain probability
        if label == 'O' and random.random() < deletion_prob:
            continue
        augmented_tokens.append(token)
        augmented_labels.append(label)

    return augmented_tokens, augmented_labels

def augment_token_swap(tokens, labels, swap_prob=0.05):
    """Swap adjacent non-entity tokens"""
    augmented_tokens = tokens.copy()

    # Find eligible positions for swapping (adjacent O tokens)
    eligible_positions = []
    for i in range(len(labels) - 1):
        if labels[i] == 'O' and labels[i + 1] == 'O':
            eligible_positions.append(i)

    # Perform swaps
    num_swaps = int(len(eligible_positions) * swap_prob)
    if eligible_positions and num_swaps > 0:
        positions_to_swap = random.sample(eligible_positions, min(num_swaps, len(eligible_positions)))
        for pos in positions_to_swap:
            augmented_tokens[pos], augmented_tokens[pos + 1] = augmented_tokens[pos + 1], augmented_tokens[pos]

    return augmented_tokens, labels

def augment_data(data, augmentation_factor=2):
    """Apply data augmentation to increase dataset size"""
    augmented_data = []

    # First add all original examples
    augmented_data.extend(data)

    print(f"\nApplying data augmentation with factor {augmentation_factor}...")
    # Generate augmented examples
    for _ in range(augmentation_factor - 1):
        for example in data:
            tokens = example['tokens']
            labels = example['labels']

            # Choose random augmentation technique
            augmentation_choice = random.choice(['replace', 'delete', 'swap', 'combined'])

            if augmentation_choice == 'replace':
                aug_tokens, aug_labels = augment_token_replacement(tokens, labels)
            elif augmentation_choice == 'delete':
                aug_tokens, aug_labels = augment_token_deletion(tokens, labels)
            elif augmentation_choice == 'swap':
                aug_tokens, aug_labels = augment_token_swap(tokens, labels)
            else:  # combined
                # Apply multiple augmentations sequentially
                aug_tokens, aug_labels = tokens.copy(), labels.copy()

                if random.random() < 0.5:
                    aug_tokens, aug_labels = augment_token_replacement(aug_tokens, aug_labels, 0.1)
                if random.random() < 0.5:
                    aug_tokens, aug_labels = augment_token_deletion(aug_tokens, aug_labels, 0.03)
                if random.random() < 0.5:
                    aug_tokens, aug_labels = augment_token_swap(aug_tokens, aug_labels, 0.03)

            # Create augmented example
            augmented_data.append({
                'tokens': aug_tokens,
                'labels': aug_labels,
                'original_text': example['original_text'],
                'entity_presence': example['entity_presence'],
                'augmented': True
            })

    return augmented_data

# APPLY AUGMENTATION ONLY TO TRAINING DATA
AUGMENTATION_FACTOR = 3  # Original data + 2x augmented data
train_data_augmented = augment_data(train_data, AUGMENTATION_FACTOR)
print(f"Training data augmentation complete. Training set size increased from {len(train_data)} to {len(train_data_augmented)}")



Applying data augmentation with factor 3...
Training data augmentation complete. Training set size increased from 301 to 903


In [ ]:

# Count entity presence distribution after augmentation for training data
train_entity_presence_counts = Counter(ex['entity_presence'] for ex in train_data_augmented)
print("\nTraining data entity distribution after augmentation:")
for entity_type, count in train_entity_presence_counts.most_common():
    print(f"{entity_type}: {count}")


Training data entity distribution after augmentation:
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 405
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME: 87
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 63
COLLEGE NAME+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 36
COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 30
COLLEGE NAME+COMPANY+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 30
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+NAME+SKILLS: 27
COLLEGE NAME+COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 24
COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 21
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+NAME+SKILLS: 15
COLLEGE NAME+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 15
DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 12
COLLEGE NAME+DEGREE+DESIGNATION+LOCATION+NAME: 12
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME: 12
COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 9
COLLEGE NAME+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 9
COMPANY+DESIGNATION+EMA

In [ ]:

# Check class distribution in splits
def print_entity_distribution(dataset, name):
    entity_presence = Counter([ex['entity_presence'] for ex in dataset])
    print(f"\n{name} set entity distribution:")
    for entity_type, count in entity_presence.most_common():
        print(f"{entity_type}: {count}")

print_entity_distribution(train_data_augmented, "Training (Augmented)")
print_entity_distribution(val_data, "Validation")
print_entity_distribution(test_data, "Test")


Training (Augmented) set entity distribution:
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 405
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME: 87
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 63
COLLEGE NAME+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 36
COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 30
COLLEGE NAME+COMPANY+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 30
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+EMAIL+NAME+SKILLS: 27
COLLEGE NAME+COMPANY+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 24
COMPANY+DEGREE+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 21
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+NAME+SKILLS: 15
COLLEGE NAME+DEGREE+EMAIL+LOCATION+NAME+SKILLS: 15
DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 12
COLLEGE NAME+DEGREE+DESIGNATION+LOCATION+NAME: 12
COLLEGE NAME+COMPANY+DEGREE+DESIGNATION+LOCATION+NAME: 12
COMPANY+DEGREE+DESIGNATION+LOCATION+NAME+SKILLS: 9
COLLEGE NAME+DESIGNATION+EMAIL+LOCATION+NAME+SKILLS: 9
COMPANY+DESIGNATION+EMAIL+LOCAT

In [ ]:
# Fixed and improved ResumeNERDataset class for BERT
class ResumeNERDataset(Dataset):
    def __init__(self, texts, tags, tokenizer, max_len, label2id):
        self.texts = texts
        self.tags = tags
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label2id = label2id

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item_idx):
        text = self.texts[item_idx]
        tags = self.tags[item_idx]

        # Ensure text and tags have the same length
        if len(text) != len(tags):
            min_len = min(len(text), len(tags))
            text = text[:min_len]
            tags = tags[:min_len]

        # Tokenize with BERT tokenizer
        encoding = self.tokenizer(
            text,
            is_split_into_words=True,
            return_offsets_mapping=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        # Create label tensor with proper alignment to wordpieces
        word_ids = encoding.word_ids()
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # Special tokens like [CLS] and [SEP]
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # First token of a word
                if word_idx < len(tags):
                    label_ids.append(self.label2id.get(tags[word_idx], self.label2id['O']))
                else:
                    label_ids.append(-100)
            else:
                # Continuation of a word (WordPiece)
                if tags[previous_word_idx].startswith('B-'):
                    # Convert B- to I- for continuation tokens
                    i_tag = f"I-{tags[previous_word_idx][2:]}"
                    label_ids.append(self.label2id.get(i_tag, self.label2id['O']))
                else:
                    # Keep the same tag for other cases
                    label_ids.append(self.label2id.get(tags[previous_word_idx], self.label2id['O']))

            previous_word_idx = word_idx

        # Create final item with all tensors
        item = {key: val.squeeze() for key, val in encoding.items() if key != 'offset_mapping'}
        item['labels'] = torch.tensor(label_ids)

        return item


In [ ]:

from transformers import BertTokenizerFast, BertForTokenClassification
# Prepare datasets with adequate max length using BERT
MAX_LEN = 512
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Use augmented training data, but original validation and test data
train_texts = [example['tokens'] for example in train_data_augmented]
train_tags = [example['labels'] for example in train_data_augmented]
val_texts = [example['tokens'] for example in val_data]
val_tags = [example['labels'] for example in val_data]
test_texts = [example['tokens'] for example in test_data]
test_tags = [example['labels'] for example in test_data]




train_dataset = ResumeNERDataset(train_texts, train_tags, tokenizer, MAX_LEN, label2id)
val_dataset = ResumeNERDataset(val_texts, val_tags, tokenizer, MAX_LEN, label2id)
test_dataset = ResumeNERDataset(test_texts, test_tags, tokenizer, MAX_LEN, label2id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

# Create data loaders with appropriate batch size for your data
BATCH_SIZE = 8  # Reduced to allow more gradient updates and better learning
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:

# Initialize BERT model
model = BertForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")
model.to(device)


Using device: cuda


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# Improved training parameters
LEARNING_RATE = 3e-5  # Slightly higher learning rate
EPOCHS = 30  # More epochs for better learning
WEIGHT_DECAY = 0.01  # Add weight decay to reduce overfitting

In [ ]:
# Configure optimizer with weight decay
optimizer = AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

In [ ]:
# Setup scheduler with warmup
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),  # 10% warmup
    num_training_steps=total_steps
)

In [ ]:
# Enhanced training function with gradient accumulation
def train(accumulation_steps=4):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc="Training")
    optimizer.zero_grad()

    for idx, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss = loss / accumulation_steps  # Normalize loss for accumulation
        loss.backward()

        total_loss += loss.item() * accumulation_steps  # Denormalize for reporting

        # Accumulated gradient update
        if (idx + 1) % accumulation_steps == 0 or (idx + 1) == len(train_dataloader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        progress_bar.set_postfix({'loss': f"{loss.item() * accumulation_steps:.4f}"})

    return total_loss / len(train_dataloader)

In [ ]:
# Change 1: Add accuracy metric to the evaluate function
def evaluate(dataloader):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []

    # Variables to track token-level accuracy
    correct_tokens = 0
    total_tokens = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            total_loss += loss.item()

            # Get predictions and true labels
            logits = outputs.logits
            preds = torch.argmax(logits, dim=2)

            # Remove ignored index (-100)
            for i in range(input_ids.shape[0]):
                pred_list = []
                true_list = []
                for j in range(input_ids.shape[1]):
                    if labels[i, j] != -100:
                        pred_list.append(id2label[preds[i, j].item()])
                        true_list.append(id2label[labels[i, j].item()])

                        # Count correct token predictions for accuracy
                        if preds[i, j].item() == labels[i, j].item():
                            correct_tokens += 1
                        total_tokens += 1

                predictions.append(pred_list)
                true_labels.append(true_list)

    # Calculate token-level accuracy
    token_accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0

    # Calculate metrics using seqeval
    report = classification_report(true_labels, predictions, output_dict=True)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    # Print token-level accuracy
    print(f"\nToken-level accuracy: {token_accuracy:.4f}")

    # Print entity-level metrics
    print("\nEntity-level metrics:")
    for entity_type, metrics in report.items():
        if entity_type not in ['micro avg', 'macro avg', 'weighted avg', 'O']:
            print(f"{entity_type}:")
            print(f"  Precision: {metrics['precision']:.4f}")
            print(f"  Recall: {metrics['recall']:.4f}")
            print(f"  F1: {metrics['f1-score']:.4f}")
            print(f"  Support: {metrics['support']}")

    return total_loss / len(dataloader), report, precision, recall, f1, token_accuracy

In [ ]:
# Change 2: Update the training loop to track accuracy
train_losses = []
val_losses = []
val_accuracies = []  # New list to track validation accuracies
best_val_f1 = 0
best_model_path = "best_resume_ner_model.pt"
patience = 4  # Early stopping patience
no_improvement = 0

In [ ]:
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    # Train
    train_loss = train()
    train_losses.append(train_loss)

    # Evaluate on validation set
    val_loss, val_report, val_precision, val_recall, val_f1, val_accuracy = evaluate(val_dataloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)  # Track validation accuracy

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")

    # Save best model and check for early stopping
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with F1: {val_f1:.4f}")
        no_improvement = 0
    else:
        no_improvement += 1
        print(f"No improvement for {no_improvement} epochs")

        if no_improvement >= patience:
            print("Early stopping triggered")
            break



Epoch 1/30


Evaluating: 100%|██████████| 8/8 [00:04<00:00,  1.61it/s]



Token-level accuracy: 0.5452

Entity-level metrics:
COLLEGE NAME:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 55
COMPANY:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 149
DEGREE:
  Precision: 0.0005
  Recall: 0.0208
  F1: 0.0010
  Support: 48
DESIGNATION:
  Precision: 0.0026
  Recall: 0.0140
  F1: 0.0044
  Support: 143
EMAIL:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 70
LOCATION:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 98
NAME:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 64
SKILLS:
  Precision: 0.0007
  Recall: 0.0132
  F1: 0.0014
  Support: 76
Train Loss: 2.8257
Val Loss: 2.3670, Accuracy: 0.5452, Precision: 0.0005, Recall: 0.0057, F1: 0.0009
New best model saved with F1: 0.0009

Epoch 2/30


Evaluating: 100%|██████████| 8/8 [00:04<00:00,  1.77it/s]



Token-level accuracy: 0.7787

Entity-level metrics:
COLLEGE NAME:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 55
COMPANY:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 149
DEGREE:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 48
DESIGNATION:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 143
EMAIL:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 70
LOCATION:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 98
NAME:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 64
SKILLS:
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
  Support: 76
Train Loss: 1.4823
Val Loss: 1.0224, Accuracy: 0.7787, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
No improvement for 1 epochs

Epoch 3/30


...


Token-level accuracy: 0.9644

Entity-level metrics:
COLLEGE NAME:
  Precision: 0.7541
  Recall: 0.8364
  F1: 0.7931
  Support: 55
COMPANY:
  Precision: 0.7000
  Recall: 0.7047
  F1: 0.7023
  Support: 149
DEGREE:
  Precision: 0.7321
  Recall: 0.8542
  F1: 0.7885
  Support: 48
DESIGNATION:
  Precision: 0.7877
  Recall: 0.8042
  F1: 0.7958
  Support: 143
EMAIL:
  Precision: 0.8889
  Recall: 0.9143
  F1: 0.9014
  Support: 70
LOCATION:
  Precision: 0.8774
  Recall: 0.9490
  F1: 0.9118
  Support: 98
NAME:
  Precision: 0.9692
  Recall: 0.9844
  F1: 0.9767
  Support: 64
SKILLS:
  Precision: 0.7067
  Recall: 0.6974
  F1: 0.7020
  Support: 76
Train Loss: 0.0087
Val Loss: 0.2100, Accuracy: 0.9644, Precision: 0.7934, Recall: 0.8250, F1: 0.8089
No improvement for 2 epochs

Epoch 28/30


Evaluating: 100%|██████████| 8/8 [00:04<00:00,  1.73it/s]



Token-level accuracy: 0.9643

Entity-level metrics:
COLLEGE NAME:
  Precision: 0.7419
  Recall: 0.8364
  F1: 0.7863
  Support: 55
COMPANY:
  Precision: 0.7000
  Recall: 0.7517
  F1: 0.7249
  Support: 149
DEGREE:
  Precision: 0.7736
  Recall: 0.8542
  F1: 0.8119
  Support: 48
DESIGNATION:
  Precision: 0.7548
  Recall: 0.8182
  F1: 0.7852
  Support: 143
EMAIL:
  Precision: 0.8451
  Recall: 0.8571
  F1: 0.8511
  Support: 70
LOCATION:
  Precision: 0.8532
  Recall: 0.9490
  F1: 0.8986
  Support: 98
NAME:
  Precision: 0.9545
  Recall: 0.9844
  F1: 0.9692
  Support: 64
SKILLS:
  Precision: 0.6463
  Recall: 0.6974
  F1: 0.6709
  Support: 76
Train Loss: 0.0086
Val Loss: 0.2174, Accuracy: 0.9643, Precision: 0.7718, Recall: 0.8321, F1: 0.8008
No improvement for 3 epochs

Epoch 29/30


Evaluating: 100%|██████████| 8/8 [00:04<00:00,  1.75it/s]



Token-level accuracy: 0.9679

Entity-level metrics:
COLLEGE NAME:
  Precision: 0.7419
  Recall: 0.8364
  F1: 0.7863
  Support: 55
COMPANY:
  Precision: 0.6879
  Recall: 0.7248
  F1: 0.7059
  Support: 149
DEGREE:
  Precision: 0.7778
  Recall: 0.8750
  F1: 0.8235
  Support: 48
DESIGNATION:
  Precision: 0.7338
  Recall: 0.7902
  F1: 0.7609
  Support: 143
EMAIL:
  Precision: 0.7412
  Recall: 0.9000
  F1: 0.8129
  Support: 70
LOCATION:
  Precision: 0.8407
  Recall: 0.9694
  F1: 0.9005
  Support: 98
NAME:
  Precision: 0.9403
  Recall: 0.9844
  F1: 0.9618
  Support: 64
SKILLS:
  Precision: 0.5870
  Recall: 0.7105
  F1: 0.6429
  Support: 76
Train Loss: 0.0107
Val Loss: 0.1891, Accuracy: 0.9679, Precision: 0.7449, Recall: 0.8307, F1: 0.7855
No improvement for 4 epochs
Early stopping triggered


In [ ]:
# Change 3: Load best model and evaluate with accuracy on test set
print("\nLoading best model for testing...")
model.load_state_dict(torch.load(best_model_path))
test_loss, test_report, test_precision, test_recall, test_f1, test_accuracy = evaluate(test_dataloader)

print("\nFinal Test Results:")
print(f"Loss: {test_loss:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1 Score: {test_f1:.4f}")



Loading best model for testing...


Evaluating: 100%|██████████| 9/9 [00:05<00:00,  1.76it/s]



Token-level accuracy: 0.9493

Entity-level metrics:
COLLEGE NAME:
  Precision: 0.7818
  Recall: 0.8269
  F1: 0.8037
  Support: 52
COMPANY:
  Precision: 0.8103
  Recall: 0.8952
  F1: 0.8507
  Support: 105
DEGREE:
  Precision: 0.9038
  Recall: 0.9216
  F1: 0.9126
  Support: 51
DESIGNATION:
  Precision: 0.8468
  Recall: 0.8607
  F1: 0.8537
  Support: 122
EMAIL:
  Precision: 0.8533
  Recall: 0.9014
  F1: 0.8767
  Support: 71
LOCATION:
  Precision: 0.8839
  Recall: 0.8839
  F1: 0.8839
  Support: 112
NAME:
  Precision: 1.0000
  Recall: 0.9706
  F1: 0.9851
  Support: 68
SKILLS:
  Precision: 0.4182
  Recall: 0.6389
  F1: 0.5055
  Support: 36

Final Test Results:
Loss: 0.3211
Accuracy: 0.9493
Precision: 0.8260
Recall: 0.8768
F1 Score: 0.8506


In [ ]:
# Change 4: Plot accuracy along with loss
plt.figure(figsize=(12, 10))

# Plot loss
plt.subplot(2, 1, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, 'b-', label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, 'r-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot accuracy
plt.subplot(2, 1, 2)
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, 'g-', label='Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_metrics.png')
plt.close()

In [ ]:
# Improved prediction function with better entity extraction
def predict_entities(text):
    """Predict NER tags for a new text with improved entity extraction"""
    model.eval()

    # Tokenize the text
    tokens = []
    for match in re.finditer(r'\S+', text):
        tokens.append(match.group())

    # Prepare input for the model
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_offsets_mapping=True,
        padding='max_length',
        truncation=True,
        max_length=MAX_LEN,
        return_tensors='pt'
    )

    # Move inputs to device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=2)

    # Convert predictions to labels
    predicted_labels = []
    word_ids = inputs.word_ids(0)  # Batch index 0
    previous_word_idx = None

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue

        if idx < len(predictions[0]):
            predicted_labels.append(id2label[predictions[0, idx].item()])
        else:
            predicted_labels.append('O')

        previous_word_idx = word_idx

    # Truncate predictions if needed
    predicted_labels = predicted_labels[:len(tokens)]

    # Combine tokens and predictions
    result = list(zip(tokens, predicted_labels))

    return result

In [ ]:
# Improved entity grouping for better visualization
def group_entities(predictions):
    """Group tokens with the same entity type together with improved boundary detection"""
    entities = {}
    current_entity = None
    current_text = []

    for token, label in predictions:
        if label == 'O':
            if current_entity:
                if current_entity not in entities:
                    entities[current_entity] = []
                entities[current_entity].append(' '.join(current_text))
                current_entity = None
                current_text = []
        elif label.startswith('B-'):
            if current_entity:
                if current_entity not in entities:
                    entities[current_entity] = []
                entities[current_entity].append(' '.join(current_text))
            current_entity = label[2:]  # Remove B- prefix
            current_text = [token]
        elif label.startswith('I-'):
            if current_entity == label[2:]:  # Only append if it's the same entity type
                current_text.append(token)
            else:
                # Start a new entity if the I- tag doesn't match current entity
                if current_entity:
                    if current_entity not in entities:
                        entities[current_entity] = []
                    entities[current_entity].append(' '.join(current_text))
                current_entity = label[2:]
                current_text = [token]

    # Add the last entity if there is one
    if current_entity and current_text:
        if current_entity not in entities:
            entities[current_entity] = []
        entities[current_entity].append(' '.join(current_text))

    return entities

In [ ]:
# Example prediction with more comprehensive resume text
example_text = """John Smith
Email: john.smith@example.com
Phone: +1-234-567-8901
LinkedIn: linkedin.com/in/johnsmith

A passionate software engineer with 3 years of experience in full-stack development. Proficient in JavaScript, React, Node.js, and Python.

Education
B.Tech in Computer Science, Indian Institute of Technology Madras, 2018 - 2022
CGPA: 8.7/10

Experience
Software Engineer at Google, Bangalore — Jul 2022 to Present
- Built scalable REST APIs using Node.js and Express
- Led a team of 3 in migrating the dashboard to React

Software Engineering Intern at Microsoft, Hyderabad — May 2021 to Jul 2021
- Developed internal tools using Python and Flask
- Improved application load time by 30%

Projects
Resume Parser using BERT — Built a tool to extract structured information from resumes
E-commerce Web App — MERN stack application with authentication and payment gateway integration

Skills
JavaScript, React, Node.js, Express, Python, Flask, MongoDB, PostgreSQL, Git, Docker
"""

In [ ]:
print("\nRunning prediction on comprehensive example:")
print(f"Text: {example_text[:100]}...")
predictions = predict_entities(example_text)


Running prediction on comprehensive example:
Text: John Smith
Email: john.smith@example.com
Phone: +1-234-567-8901
LinkedIn: linkedin.com/in/johnsmith
...


In [ ]:
print("\nPredicted Entities (first 20):")
for token, label in predictions[:20]:
    print(f"{token} -> {label}")


Predicted Entities (first 20):
John -> B-NAME
Smith -> I-NAME
Email: -> O
john.smith@example.com -> B-NAME
Phone: -> I-EMAIL
+1-234-567-8901 -> I-EMAIL
LinkedIn: -> O
linkedin.com/in/johnsmith -> I-EMAIL
A -> O
passionate -> O
software -> O
engineer -> O
with -> O
3 -> O
years -> O
of -> O
experience -> O
in -> O
full-stack -> O
development. -> O


In [ ]:
print("\nGrouped Entities:")
grouped = group_entities(predictions)
for entity_type, mentions in grouped.items():
    print(f"{entity_type}: {mentions}")


Grouped Entities:
NAME: ['John Smith', 'john.smith@example.com']
EMAIL: ['Phone: +1-234-567-8901', 'linkedin.com/in/johnsmith']
SKILLS: ['JavaScript,', 'React,', 'Node.js,', 'Python.', 'JavaScript, React, Node.js, Express, Python, Flask, MongoDB, PostgreSQL, Git, Docker']
DEGREE: ['B.Tech in Computer Science,']
COLLEGE NAME: ['Indian Institute of Technology Madras,']
DESIGNATION: ['Software Engineer', 'Software Engineering']
